# Step 3: Feature extraction (image → vector)

Implement in src/features.py later
### Techniques:
- Color histogram → simple RGB histogram vector
- Texture features → Local Binary Patterns (LBP)
- Shape/structure features → Histogram of Oriented Gradients (HOG)
### Output: 1D fixed-length feature vector for each image.
- Save extracted features (e.g., numpy arrays) to avoid re-computation.

> Raw images are 2D (height × width × channels).
SVM and kNN require 1D vectors per sample.
Features should capture color, shape, and texture of trash items, which helps the classifier distinguish metal, paper, plastic, etc.

+ If a sample has high values in the LBP section corresponding to “rough texture” → likely paper.
+ If it has high values in HOG section corresponding to vertical edges → maybe a bottle.
+ If it has high blue in color section → maybe a plastic bottle.

In [27]:
import cv2
import numpy as np
from skimage.feature import hog
from skimage.color import rgb2gray
from skimage.feature import local_binary_pattern
import os
import pandas as pd

In [28]:
# Color histogram
def color_histogram(img, bins=32):
    # img: RGB image
    hist_r = cv2.calcHist([img], [0], None, [bins], [0, 256]) # Red
    hist_g = cv2.calcHist([img], [1], None, [bins], [0, 256]) # Green
    hist_b = cv2.calcHist([img], [2], None, [bins], [0, 256]) # Blue
    hist = np.concatenate([hist_r, hist_g, hist_b]).flatten()
    hist = hist / np.sum(hist)  # normalize
    return hist


This function computes HOG features for an image. HOG is a powerful feature descriptor often used in object detection and image classification, because it captures shape and edge information rather than just color.
It’s especially useful when the structure of objects (like the contours of waste items) is more important than their color.

In [29]:
# HOG features
def hog_features(img, pixels_per_cell=(16,16), cells_per_block=(2,2)):
    gray = rgb2gray(img)
    features = hog(gray,
                   pixels_per_cell=pixels_per_cell,
                   cells_per_block=cells_per_block,
                   block_norm='L2-Hys')
    return features



In [30]:
def lbp_features(img, P=8, R=1):
    # Convert to grayscale
    gray = rgb2gray(img)


    # Compute LBP
    lbp = local_binary_pattern(gray, P, R, method='uniform')

    # Build histogram of LBP
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp, bins=n_bins, range=(0, n_bins), density=True)

    return hist

| Feature type    | Vector length                             | What it represents              |
| --------------- | ----------------------------------------- | ------------------------------- |
| Color histogram | 96 (32 bins × 3 channels)                 | Color distribution of the image |
| HOG             | 500+ (depends on image size & HOG params) | Edge/shape patterns             |
| LBP             | 59 (uniform patterns for P=8)             | Local texture patterns          |


- Concatenated vector length ≈ 96 + 500 + 59 = 655 (example).

In [31]:
def extract_features(img):
    color_feat = color_histogram(img)
    hog_feat = hog_features(img)
    lbp_feat = lbp_features(img)

    # Concatenate all features into a single vector
    features = np.concatenate([color_feat, hog_feat, lbp_feat])
    return features

# Output Sample

In [32]:
# --- Path to images ---
image_folder = "/Users/rodynaamr/Image_Classification_SVM_kNN/data/paper"
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Limit to first 10 images
image_files = image_files[:10]

# --- Extract features ---
feature_matrix = []
for file in image_files:
    path = os.path.join(image_folder, file)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    features = extract_features(img)
    feature_matrix.append(features)

feature_matrix = np.array(feature_matrix)
print("Feature matrix shape:", feature_matrix[1])

/Users/rodynaamr/Image_Classification_SVM_kNN/.venv/lib/python3.12/site-packages/skimage/color/colorconv.py:984: RuntimeWarning: divide by zero encountered in matmul
  return rgb @ coeffs
/Users/rodynaamr/Image_Classification_SVM_kNN/.venv/lib/python3.12/site-packages/skimage/color/colorconv.py:984: RuntimeWarning: overflow encountered in matmul
  return rgb @ coeffs
/Users/rodynaamr/Image_Classification_SVM_kNN/.venv/lib/python3.12/site-packages/skimage/color/colorconv.py:984: RuntimeWarning: invalid value encountered in matmul
  return rgb @ coeffs
/Users/rodynaamr/Image_Classification_SVM_kNN/.venv/lib/python3.12/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


Feature matrix shape: [0.00000000e+00 8.47710544e-06 2.37358938e-04 ... 5.96822103e-02
 4.57326253e-01 4.71038818e-02]


# add vectors to a csv file, create a table for each waste type with the data from both folder with images augmented and not

In [33]:
# --- Paths ---
base_path = "/Users/rodynaamr/Image_Classification_SVM_kNN/data"
waste_types = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]

In [34]:
all_features = []
all_labels = []

for waste in waste_types:
    for folder_suffix in ["", "_aug"]:
        folder_path = os.path.join(base_path, waste + folder_suffix)
        if not os.path.exists(folder_path):
            continue

        for filename in os.listdir(folder_path):
            if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
                continue

            img_path = os.path.join(folder_path, filename)

            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            feats = extract_features(img)
            all_features.append(feats)
            all_labels.append(waste)

# Convert to numpy
X = np.array(all_features)
y = np.array(all_labels)


# Encode Labels + Scale Features


In [35]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Encode labels to integers
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# check feature matrix


In [36]:
import numpy as np

print("Any NaN?:", np.isnan(X).any())
print("Any inf?:", np.isinf(X).any())
print("Max value:", np.max(X))
print("Min value:", np.min(X))


Any NaN?: False
Any inf?: False
Max value: 0.9999999987499999
Min value: 0.0


# save data as npy for faster access
> how to load it later
> - import numpy as np
> - X = np.load("X.npy")
> - y = np.load("y.npy")


In [37]:
np.save("features.npy", X_scaled)
np.save("labels.npy", y_encoded)
